# Desktop DNS Notebook 4

## RANS modelling



In [1]:
# reload modules
%reload_ext autoreload
%autoreload 2


In [15]:
### Create RANS mesh

import matplotlib.pyplot as plt
import timeit

from meshing.read_profile   import *
from meshing.blade_mesher import *
from meshing.read_case import *

# pop-out figure
%matplotlib qt

# mesh refinement inputs
refine_fac = 0.5 # downsample for RANS 
ywall = 0.01/150 # the final near wall cell height
msmooths = 100 # number of iterations for the poisson solver

# set case names
old_casename = 'notebook3'
new_casename = 'notebook4'

# read old casefiles
case = read_case(old_casename)

# case name and profile coordinates
casename = new_casename
case['casename'] = casename
mesh   = {}
mesh['case'] = casename
mesh['profile'] = 'geom/profile.txt'

# initial mesh inputs
mesh['npp'] = 2*[None]
mesh['ywall'] = 2*[None]
mesh['msmooths'] = 2*[None]

mesh['npp'][0] = 4
mesh['Lup'] = 1.0
mesh['Ldn'] = 1.0
mesh['Lo'] = 0.2
mesh['ywall'][0] = 0.001
mesh['msmooths'][0] = 500
mesh['cor_fac'] = 0.75 # factor shrinking multi-block corner cells - lower values give more shrinkage.

# final mesh inputs
mesh['npp'][1] = 28 # sets minimum points in any direction in smallest block
mesh['refine_fac'] = refine_fac 
mesh['ywall'][1] =  ywall
mesh['msmooths'][1] = msmooths

# scale factor if used (set to 1.0 if not)
mesh['scale_factor'] = 1.0

# flip y coordinates
mesh['flip'] = 1

# modify solver inputs 
solver = case['solver']
solver['niter'] = 500000 # run for a few iterations to check all ok
solver['irestart'] = 1 #  restart from initial guess
solver['istats'] = 0 # switch-off statistics
solver['span'] = 0.1 # spanwise extent not used for 2D
solver['nk'] = 1 # set to 1 for 2D 
bcs = case['bcs']
gas = case['gas']

# switch-off inlet perturbations
bcs['aturb'] = 0.0 # amplification factor for turbulence

# create mesh and case files
start = timeit.default_timer()
blk = blade_mesher(mesh,bcs,gas,solver)
stop = timeit.default_timer()

print('Time: ', stop - start)  



gpu input file found
2
Writing inputs for 3dns_gpu
Total ij points
28224
Total GPUs
1
Time:  106.72818918299163


In [ ]:
## Run initial guess

In [282]:
#%reload_ext autoreload
#%autoreload 2
from meshing.initial_guess import *
from meshing.write_flo import *
from meshing.read_case import *

casename = 'notebook4'

# read casefiles
case = read_case(casename)

# create initial guess (last two integers define the streamline curvature grid dims)
prop = initial_guess(mesh,blk,bcs,gas,30,20)

# write initial flow files
write_flo(prop,casename)

gpu input file found
2
gpu input file found
2


In [6]:
### Determine eddy viscosity

from postpro.read_2d_mean import *
from postpro.plot_flo import *
from meshing.write_flo import *

prop_dns,geom_dns=read_2d_mean('../as2375_2_highre/budgets/nk120/',[1000])
#
## write instantaneous flow as re-start
#interp_flo('notebook2','notebook4'):


[0.0130743  0.0130743  0.0130743  0.00653715] 0.04576005 [1 2 3 4]
gpu input file found
2


In [174]:
plot_flo(prop_dns,geom_dns,'mut_opt',[0,100])


<module 'matplotlib.pyplot' from '/home/aw329/.local/lib/python3.6/site-packages/matplotlib/pyplot.py'>

In [142]:

### Create prescribed eddy viscosity

from postpro.write_mut import *
from postpro.read_flo import *

write_mut(prop_dns,geom_dns,'notebook4')


gpu input file found
2


In [53]:
### Run a prescribed eddy viscosity RANS simulation

In [3]:
from postpro.read_flo import *
from postpro.plot_flo import *


# pop-out figure
%matplotlib qt

# read flo
casename = 'notebook4'

prop,geom=read_flo(casename)

# plot flo
plot_flo(prop,geom,'mut_model',[0,100])

gpu input file found
2


<module 'matplotlib.pyplot' from '/home/aw329/.local/lib/python3.6/site-packages/matplotlib/pyplot.py'>

In [4]:

# plot flo
plot_flo(prop,geom,'mach',[0,0.3])
#plot_flo(prop,geom,'p',[0.96e5,1.0e5])

<module 'matplotlib.pyplot' from '/home/aw329/.local/lib/python3.6/site-packages/matplotlib/pyplot.py'>

In [7]:
import matplotlib.pyplot as plt

wall_blocks = [3,4,5,7]

plt.figure(1)
for i in wall_blocks:
    plt.plot(geom[i-1]['x'][:,-1],prop[i-1]['p'][:,-1,0],'r.')
    plt.plot(geom_dns[i-1]['x'][:,-1],prop_dns[i-1]['p'][:,-1],'k.')
plt.legend(['rans','dns'])
plt.show()

In [305]:
from postpro.plot_monitor import *

plot_monitor(casename);

gpu input file found
2
